# TelFit demo

In [ ]:
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format='retina'

### Read in an example data file that has telluric absorption

In [ ]:
from muler.hpf import HPFSpectrum

In [ ]:
file = '../../muler_example_data/HPF/01_A0V_standards/Goldilocks_20210801T083618_v1.0_0036.spectra.fits'
data = HPFSpectrum(file=file, order=27)

In [ ]:
data = data.remove_nans().trim_edges()
data = data.sky_subtract(method='vector').deblaze().normalize()
data = data.flatten_by_black_body(9_700)

In [ ]:
peak_values = np.percentile(data.flux, 90)
data = data.divide(peak_values, handle_meta='ff')

In [ ]:
ax = data.plot()
ax.axhline(1.0, linestyle='dashed', color='k')
ax.set_ylim(0);

Looks good!  What were the atmospheric conditions on that night?

In [ ]:
temp = data.meta['header']['ENVTEM']
humidity = data.meta['header']['ENVHUM']

In [ ]:
from astropy.units import temperature

In [ ]:
temp*u.imperial.deg_F

In [ ]:
temp_K = (temp*u.imperial.deg_F).to(u.Kelvin, equivalencies=temperature())

In [ ]:
temp_K

In [ ]:
humidity

### Read in a TelFit model

In [ ]:
from gollum.telluric import TelFitSpectrum

In [ ]:
path = '/Volumes/pecos/libraries/raw/telfit/grid_v1p0/'
fn = 'telfit_800_1300nm_temp294_hum060.txt'

In [ ]:
telluric_model = TelFitSpectrum(path=path+fn)

In [ ]:
telluric_model = telluric_model.air_to_vacuum()

In [ ]:
telluric_model = telluric_model.instrumental_broaden(resolving_power=55_000)

In [ ]:
telluric_model = telluric_model.resample(data)

In [ ]:
ax = telluric_model.rv_shift(-7).plot(label='Representative TelFit model', alpha=0.5)
data.plot(ax=ax, label='HPF A0V calibrator', alpha=0.5)
ax.set_ylim(0)
ax.axhline(1.0, linestyle='dotted', color='k')
ax.legend();

Neat!  Looks like the wavelengths are shifted slightly, possibly due to air/vacuum assumption differences.

In [ ]:
telluric_model.rv_shift(-7)

In [ ]:
residual = (data - telluric_model.rv_shift(-7).resample(data))*100.0

ax = residual.plot(ylo=-20, yhi=20)
ax.axhline(0, linestyle='dashed', color='k')
ax.set_ylabel('Residual (%)');

It appears that most of the residual structure stems from velocity differences in the model and data.

In [ ]:
residual = (data - telluric_model.rv_shift(-8.4).resample(data))*100.0

ax = residual.plot(ylo=-20, yhi=20)
ax.axhline(0, linestyle='dashed', color='k')
ax.set_ylabel('Residual (%)');

A 1.4 km/s speed difference seems physically implausible, which makes me think something is funky with either TelFit, Goldilocks, muler, or gollum!

### Sensitivity analysis

How do the lines vary with our parameters of interest?

In [ ]:
fn = 'telfit_800_1300nm_temp298_hum060.txt'
telluric_model2 = TelFitSpectrum(path=path+fn)

In [ ]:
telluric_model2 = telluric_model2.air_to_vacuum()
telluric_model2 = telluric_model2.instrumental_broaden(resolving_power=55_000).resample(data)

jacobian = (telluric_model2 - telluric_model) * 100

In [ ]:
ax = jacobian.plot()
ax.set_ylim(-20, 20)
ax.set_ylabel('dTrans / dTemp (%/4K)');
ax.set_title('Percent change in transmission for a 4 K Temperature change');

In [ ]:
fn = 'telfit_800_1300nm_temp294_hum065.txt'
telluric_model3 = TelFitSpectrum(path=path+fn)

In [ ]:
telluric_model3 = telluric_model3.air_to_vacuum()
telluric_model3 = telluric_model3.instrumental_broaden(resolving_power=55_000).resample(data)

jacobian = (telluric_model3 - telluric_model) *100

In [ ]:
ax = jacobian.plot()
ax.set_ylim(-20, 20)
ax.set_ylabel('dTrans / dHumidity (% per 5%)');
ax.set_title('Percent change in transmission for a 5 % Humidity change');

This wavelength region stems from $O_2$ and not $H_2O$, so I think the effect we are seeing here may stem from second order effects such as air versus self broadening parameters?  These look mostly like line width changes...  Not Sure!